In [1]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
tokenizer = AutoTokenizer.from_pretrained("robert-finetuned-pharma")
model = AutoModelForSequenceClassification.from_pretrained("robert-finetuned-pharma")

In [3]:
!mkdir roberta-results

In [9]:
sigmoid = torch.nn.Sigmoid()
for file in ["test/ntcir17_mednlp-sc_sm_ja_test.csv", "test/ntcir17_mednlp-sc_sm_en_test.csv",
             "test/ntcir17_mednlp-sc_sm_de_test.csv", "test/ntcir17_mednlp-sc_sm_fr_test.csv"]:
    test_df = pd.read_csv(file)
    for i in range(test_df.shape[0]):
        text = test_df.iloc[i,1]
        encoding = tokenizer(text, return_tensors="pt")
        encoding = {k: v.to(model.device) for k,v in encoding.items()}
        outputs = model(**encoding)
        probs = sigmoid(outputs.logits.squeeze().cpu())
        predictions = np.zeros(probs.shape)
        predictions[np.where(probs >= 0.5)] = 1
        test_df.iloc[i,2:] = predictions
    test_df.to_csv("roberta-results/" + file, index=False)